In [75]:
import requests
import pandas as pd
from IPython.core.display import display, HTML

BASE_URL = "https://www.predictit.org/api/marketdata/markets/"
DEM_NOM_ID = 3633
REP_NOM_ID = 3653
PRES_ID = 3698
PARTY_ID = 2721

HEADERS = {'Content-Type':'application/json'}
PARAMETERS = {'id': str(id), 'responseType': 'JSON'}


def request_api(id):
    # Make request
    response = requests.get(url=BASE_URL + str(id), json=PARAMETERS, headers=HEADERS)
    df = pd.DataFrame(response.json())
    df = pd.concat([df.drop(['contracts', 'shortName', 'name', 'image'], axis=1), df['contracts'].apply(pd.Series)], axis=1)
    return df


In [76]:
def get_current_cands():
    # Get presidential candidate list, add new columns with defaults
    pres_candidates = request_api(PRES_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'presidentPrice'})
    pres_candidates['dem'] = False
    pres_candidates['rep'] = False
    pres_candidates['nominationPrice'] = None

    # Get party candidates
    dem_candidates = request_api(DEM_NOM_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'nominationPrice'})
    rep_candidates = request_api(REP_NOM_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'nominationPrice'})
    
    # Loop through each, merge in nomination price and set party
    for name in pres_candidates['name']:
        if name in dem_candidates['name'].unique():
            pres_candidates.loc[pres_candidates['name'] == name,'dem'] = True
            pres_candidates.loc[pres_candidates['name'] == name,'nominationPrice'] = dem_candidates.loc[dem_candidates['name'] == name,'nominationPrice'].unique()

    for name in pres_candidates['name']:
        if name in rep_candidates['name'].unique():
            pres_candidates.loc[pres_candidates['name'] == name,'rep'] = True
            pres_candidates.loc[pres_candidates['name'] == name,'nominationPrice'] = rep_candidates.loc[rep_candidates['name'] == name,'nominationPrice'].unique()
    
    # Calculate conditional probability
    pres_candidates['conditionalProbability'] = pres_candidates['presidentPrice']/pres_candidates['nominationPrice']
    
    # Drop low candidates, give dumb numbers
    pres_candidates = pres_candidates[pres_candidates['presidentPrice'] > 0.02]
    
    # Sort by Party, Conditional Probability
    pres_candidates = pres_candidates.sort_values(by=['rep', 'conditionalProbability'], ascending=False)

    return pres_candidates

In [112]:
def html_cand_table():
    df = get_current_cands()
    df['image'] = '<img src="' + df['image'].astype(str) + '>'
    df['conditionalProbability'] = df['conditionalProbability']*100
    df['conditional chance to win'] = df['conditionalProbability'].map('{:,.2f}%'.format)
    df['party'] = 'Democrat'
    df.loc[df['rep'] == True,'party'] = 'Republican'
    df = df[['name', 'party', 'conditional chance to win']]
    df = df.set_index('name')
    del df.index.name
    return df

In [113]:
df = html_cand_table()
df

,party,conditional chance to win
Donald Trump,Republican,43.59%
Mike Pence,Republican,30.00%
Bernie Sanders,Democrat,83.33%
Joe Biden,Democrat,76.00%
Beto O'Rourke,Democrat,75.00%
Elizabeth Warren,Democrat,66.67%
Kamala Harris,Democrat,58.82%
Amy Klobuchar,Democrat,50.00%
Cory Booker,Democrat,42.86%


In [103]:
df.set_index('name')

,image,party,chance to win given their nomination
name,,,
Donald Trump,"<img src=""https://az620379.vo.msecnd.net/image...",Republican,43.59%
Mike Pence,"<img src=""https://az620379.vo.msecnd.net/image...",Republican,30.00%
Bernie Sanders,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,86.96%
Joe Biden,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,76.00%
Beto O'Rourke,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,70.59%
Elizabeth Warren,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,66.67%
Kamala Harris,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,58.82%
Amy Klobuchar,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,50.00%
Cory Booker,"<img src=""https://az620379.vo.msecnd.net/image...",Democrat,42.86%


In [1]:
import pandas as pd
import requests as req

BASE_URL = "https://www.predictit.org/api/marketdata/markets/"
DEM_NOM_ID = 3633
REP_NOM_ID = 3653
PRES_ID = 3698
PARTY_ID = 2721

HEADERS = {'Content-Type':'application/json'}
PARAMETERS = {'id': str(id), 'responseType': 'JSON'}


def request_api(id):
    # Make request
    response = req.get(url=BASE_URL + str(id), json=PARAMETERS, headers=HEADERS)
    df = pd.DataFrame(response.json())
    df = pd.concat([df.drop(['contracts', 'shortName', 'name', 'image'], axis=1), df['contracts'].apply(pd.Series)], axis=1)
    return df

def get_current_cands():
    # Get presidential candidate list, add new columns with defaults
    pres_candidates = request_api(PRES_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'presidentPrice'})
    pres_candidates['dem'] = False
    pres_candidates['rep'] = False
    pres_candidates['nominationPrice'] = None

    # Get party candidates
    dem_candidates = request_api(DEM_NOM_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'nominationPrice'})
    rep_candidates = request_api(REP_NOM_ID)[['name','image','lastTradePrice']].rename(columns={'lastTradePrice':'nominationPrice'})

    # Loop through each, merge in nomination price and set party
    for name in pres_candidates['name']:
        if name in dem_candidates['name'].unique():
            pres_candidates.loc[pres_candidates['name'] == name,'dem'] = True
            pres_candidates.loc[pres_candidates['name'] == name,'nominationPrice'] = dem_candidates.loc[dem_candidates['name'] == name,'nominationPrice'].unique()

    for name in pres_candidates['name']:
        if name in rep_candidates['name'].unique():
            pres_candidates.loc[pres_candidates['name'] == name,'rep'] = True
            pres_candidates.loc[pres_candidates['name'] == name,'nominationPrice'] = rep_candidates.loc[rep_candidates['name'] == name,'nominationPrice'].unique()

    # Calculate conditional probability
    pres_candidates['conditionalProbability'] = pres_candidates['presidentPrice']/pres_candidates['nominationPrice']

    return pres_candidates

In [2]:
get_current_cands()

,name,image,presidentPrice,dem,rep,nominationPrice,conditionalProbability
0,Donald Trump,https://az620379.vo.msecnd.net/images/Contract...,0.35,False,True,0.78,0.448718
1,Joe Biden,https://az620379.vo.msecnd.net/images/Contract...,0.20,True,False,0.24,0.833333
2,Bernie Sanders,https://az620379.vo.msecnd.net/images/Contract...,0.19,True,False,0.23,0.826087
3,Kamala Harris,https://az620379.vo.msecnd.net/images/Contract...,0.11,True,False,0.17,0.647059
4,Beto O'Rourke,https://az620379.vo.msecnd.net/images/Contract...,0.11,True,False,0.17,0.647059
5,Elizabeth Warren,https://az620379.vo.msecnd.net/images/Contract...,0.04,True,False,0.07,0.571429
6,Cory Booker,https://az620379.vo.msecnd.net/images/Contract...,0.03,True,False,0.07,0.428571
7,Mike Pence,https://az620379.vo.msecnd.net/images/Contract...,0.03,False,True,0.1,0.3
8,Amy Klobuchar,https://az620379.vo.msecnd.net/images/Contract...,0.03,True,False,0.06,0.5
9,John Kasich,https://az620379.vo.msecnd.net/images/Contract...,0.02,False,True,0.05,0.4


In [3]:
df = get_current_cands()
for c in df:
    print(c)

name
image
presidentPrice
dem
rep
nominationPrice
conditionalProbability
